In [17]:
import simpy
import pandas as pd
import random
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# ETAPA 1: EXTRAÇÃO DE DADOS

def get_process_times_from_csv(arquivo):
    """Lê um arquivo CSV Yamazumi, aplica um fator de calibração e extrai os tempos."""
    def converter_tempo_para_segundos(tempo_str):
        if pd.isna(tempo_str) or not isinstance(tempo_str, str): return 0
        parts = str(tempo_str).split(':')
        try:
            if len(parts) == 3: h, m, s = map(int, parts); return h * 3600 + m * 60 + s
            if len(parts) == 2: m, s = map(int, parts); return m * 60 + s
        except (ValueError, TypeError): return 0
        return 0

    try:
        if not os.path.exists(arquivo): return None
        df = pd.read_csv(arquivo, header=5, sep=';', encoding='latin1', on_bad_lines='skip')
        class_col = next((col for col in df.columns if 'Classifica' in col), None)
        if not class_col: return None
        total_row = df[df[class_col] == 'Total'].copy()
        if total_row.empty: return None

        start_col = df.columns.get_loc(class_col) + 1
        end_col = len(df.columns)
        try: end_col = df.columns.get_loc('Coluna1')
        except KeyError: pass
            
        colunas_postos = df.columns[start_col:end_col]
        tempos_totais = total_row[colunas_postos].dropna(axis=1, how='all')
        tempos_formatados = tempos_totais.melt(var_name='Posto', value_name='Tempo_str')
        
        # Aplica o fator de calibração aqui
        tempos_formatados['Tempo_segundos'] = tempos_formatados['Tempo_str'].apply(converter_tempo_para_segundos)
        
        return pd.Series(tempos_formatados.Tempo_segundos.values, index=tempos_formatados.Posto).to_dict()

    except Exception: return None

# --- PARÂMETROS GLOBAIS ---
# FATOR_DE_CALIBRACAO = 0.15 
NUM_OPERADORES_PERNA1 = 14  
NUM_OPERADORES_PERNA2 = 11  
HORAS_SIMULACAO = 8
META_PRODUCAO = 76
HORAS_META = 7.7 
TAKT_TIME_SEGUNDOS = 5.5 * 60



MODELOS_CSV ={
    "Accelo": {"baumuster": ["C951102", "C951104", "C951111"], "perna": 1, "tempos": "Yamazumi - Accelo.csv"},
    "Atego": {"baumuster":["C951500", "C951501", "C951511", "C951514", "C951530", "C951544"],"perna": 1, "tempos":"Yamazumi - Atego.csv"},
    "Atego (ATP)":{"baumuster": ["C968403", "C968114"], "perna": 1, "tempos":"Yamazumi - ATP.csv"}, 
    "Actros":{"baumuster": ["C963400", "C963403", "C963411", "C963414", "C963424", "C963425"], "perna": 2, "tempos":"Yamazumi - Actros.csv"},
    "Arocs": {"baumuster":["C964016", "C964216", "C964231", "C964416"], "perna": 2, "tempos":"Yamazumi - Arocs.csv"},
    "Axor (ATP +)":{"baumuster": ["C968150", "C968450", "C968453", "C968461", "C968475"], "perna":2, "tempos": "Yamazumi - Actros.csv"}
}

# MODELOS_CSV = {
#   "Atego": "Yamazumi - Atego.csv", "ATP": "Yamazumi - ATP.csv",
#     "Actros": "Yamazumi - Actros.csv", "Arocs": "Yamazumi - Arocs.csv",
# }

print("--- Carregando e Calibrando Dados de Processo ---")
#PROCESSOS_FABRICA = {modelo: get_process_times_from_csv(arquivo) for modelo, arquivo in MODELOS_CSV[modelo]["tempos"]}
PROCESSOS_FABRICA = {
    modelo: get_process_times_from_csv(dados["tempos"])
    for modelo, dados in MODELOS_CSV.items()
}
PROCESSOS_FABRICA = {k: v for k, v in PROCESSOS_FABRICA.items() if v}
print(PROCESSOS_FABRICA)
print(f"Dados para {len(PROCESSOS_FABRICA.keys())} modelos carregados com sucesso.")

print(PROCESSOS_FABRICA)

--- Carregando e Calibrando Dados de Processo ---
{'Accelo': {'Arrefec.': 316, 'Diesel': 253, 'Reaperto': 235, 'Estepe': 290, 'Pneu LD': 250, 'Pneu LE': 250, 'Aperto LD': 304, 'Aperto LE': 304, 'Grade': 270, 'MecÃ¢nica 1': 194, 'MecÃ¢nica 2': 194, 'ElÃ©trica 1 ': 357, 'ElÃ©trica 2': 357, 'Controle ': 265, 'Motorista': 266, 'Quis': 270}, 'Atego': {'Arrefec.': 314, 'Diesel': 314, 'Reaperto': 294, 'Estepe': 267, 'Pneu LD': 308, 'Pneu LE': 308, 'Aperto LD': 315, 'Aperto LE': 315, 'Grade': 234, 'MecÃ¢nica 1': 194, 'MecÃ¢nica 2': 194, 'ElÃ©trica 1 ': 360, 'ElÃ©trica 2': 360, 'Controle ': 275, 'Motorista': 266, 'Quis': 270}, 'Atego (ATP)': {'Arrefec.': 330, 'Diesel': 326, 'Reaperto': 557, '5Âª Roda': 724, 'Estepe': 319, 'Pneu LD': 256, 'Pneu LE': 256, 'Aperto LD': 323, 'Aperto LE': 323, 'Grade': 230, 'MecÃ¢nica 1': 194, 'MecÃ¢nica 2': 194, 'ElÃ©trica 1 ': 360, 'ElÃ©trica 2': 360, 'Controle ': 275, 'Motorista': 266, 'Quis': 270}, 'Actros': {'Abastecimento': 555, 'Passa-disÃ§o': 752, 'Chineleir

In [18]:
# Carregar tempos de processo
def carregar_tempos_processos():
    return {
        modelo: get_process_times_from_csv(dados["tempos"])
        for modelo, dados in MODELOS_CSV.items()
        if get_process_times_from_csv(dados["tempos"])
    }

PROCESSOS_FABRICA = carregar_tempos_processos()
print(f"Dados para {len(PROCESSOS_FABRICA.keys())} modelos carregados com sucesso.")

# Função para formatar o tempo
def format_time(seconds):
    return f"{int(seconds//3600):02}:{int((seconds%3600)//60):02}:{int(seconds%60):02}"

# Função que retorna tempos por posto

def obter_tempos_por_posto(modelo):
    if modelo not in PROCESSOS_FABRICA:
        raise ValueError(f"Modelo {modelo} não encontrado nos dados.")
    return PROCESSOS_FABRICA[modelo]

# Processo de um caminhão

# Dicionário global para registrar uso de postos
uso_postos = {}

# Processo de um caminhão com monitoramento de eficiência
def caminhao(env, id_caminhao, modelo, baumuster, perna, resultados):
    tempos_postos = obter_tempos_por_posto(modelo)
    tempo_entrada = env.now

    for posto, tempo in tempos_postos.items():
        inicio = env.now
        yield env.timeout(tempo)
        fim = env.now

        # Monitorar tempo de uso do posto
        if posto not in uso_postos:
            uso_postos[posto] = {"tempo_uso": 0, "ocupacoes": 0}
        uso_postos[posto]["tempo_uso"] += (fim - inicio)
        uso_postos[posto]["ocupacoes"] += 1

    tempo_saida = env.now
    turno = int(tempo_saida // (8 * 3600)) + 1  # Cada turno tem 8 horas
    resultados.append({
        "ID": id_caminhao,
        "Modelo": modelo,
        "Baumuster": baumuster,
        "Perna": perna,
        "Tempo_saida_s": tempo_saida,
        "Turno": turno
    })

# Gera caminhões periodicamente

def gerador_perna(env, resultados, takt_time, modelos_perna, perna_nome):
    id_caminhao = 0
    while True:
        id_caminhao += 1
        modelo = random.choice(list(modelos_perna.keys()))
        baumuster = random.choice(modelos_perna[modelo])
        print(f"[{format_time(env.now)}] CHEGADA: Caminhão_{id_caminhao} ({modelo}, {baumuster}) → {perna_nome}")
        env.process(caminhao(env, id_caminhao, modelo, baumuster, perna_nome, resultados))
        yield env.timeout(takt_time)




Dados para 6 modelos carregados com sucesso.


In [20]:
# Executa a simulação e gera relatórios
def executar_simulacao():
    env = simpy.Environment()
    resultados = []
    # Define os modelos por perna
    modelos_perna1 = {k: v["baumuster"] for k, v in MODELOS_CSV.items() if v["perna"] == 1}
    modelos_perna2 = {k: v["baumuster"] for k, v in MODELOS_CSV.items() if v["perna"] == 2}

# Inicia os dois geradores separados
    env.process(gerador_perna(env, resultados, 5.5 * 60, modelos_perna1, "Perna 1"))
    env.process(gerador_perna(env, resultados, 15 * 60, modelos_perna2, "Perna 2"))

    env.run(until=HORAS_SIMULACAO * 3600)
    df = pd.DataFrame(resultados)

    # --- Relatório 1: Caminhões por modelo por turno ---
    print("\n🚚 Caminhões produzidos por turno:")
    producao_por_turno = df.groupby(['Turno', 'Modelo']).size().unstack(fill_value=0)
    print(producao_por_turno)

    # --- Relatório 2: Eficiência dos postos ---
    print("\n⚙️ Eficiência por posto (tempo de uso / tempo total disponível):")
    tempo_total = HORAS_SIMULACAO * 3600
    eficiencia_postos = []
    for posto, dados in uso_postos.items():
        eficiencia = (dados["tempo_uso"] / tempo_total) * 100
        eficiencia_postos.append({
            "Posto": posto,
            "Tempo total utilizado (s)": round(dados["tempo_uso"], 2),
            "Ocupações": dados["ocupacoes"],
            "Eficiência (%)": round(eficiencia, 2)
        })
    df_ef = pd.DataFrame(eficiencia_postos).sort_values(by="Eficiência (%)", ascending=False)
    print(df_ef)

    return df, df_ef, producao_por_turno

# Execução principal
if __name__ == "__main__":
    df_resultados, df_eficiencia, df_turnos = executar_simulacao()

    print("\n📊 Primeiros resultados:")
    print(df_resultados.head())


[00:00:00] CHEGADA: Caminhão_1 (Atego, C951511) → Perna 1
[00:00:00] CHEGADA: Caminhão_1 (Actros, C963424) → Perna 2
[00:05:30] CHEGADA: Caminhão_2 (Accelo, C951104) → Perna 1
[00:11:00] CHEGADA: Caminhão_3 (Atego, C951500) → Perna 1
[00:15:00] CHEGADA: Caminhão_2 (Axor (ATP +), C968453) → Perna 2
[00:16:30] CHEGADA: Caminhão_4 (Atego (ATP), C968114) → Perna 1
[00:22:00] CHEGADA: Caminhão_5 (Atego (ATP), C968403) → Perna 1
[00:27:30] CHEGADA: Caminhão_6 (Atego (ATP), C968403) → Perna 1
[00:30:00] CHEGADA: Caminhão_3 (Arocs, C964231) → Perna 2
[00:33:00] CHEGADA: Caminhão_7 (Atego (ATP), C968114) → Perna 1
[00:38:30] CHEGADA: Caminhão_8 (Atego (ATP), C968403) → Perna 1
[00:44:00] CHEGADA: Caminhão_9 (Atego, C951530) → Perna 1
[00:45:00] CHEGADA: Caminhão_4 (Actros, C963411) → Perna 2
[00:49:30] CHEGADA: Caminhão_10 (Atego (ATP), C968114) → Perna 1
[00:55:00] CHEGADA: Caminhão_11 (Atego (ATP), C968403) → Perna 1
[01:00:00] CHEGADA: Caminhão_5 (Actros, C963414) → Perna 2
[01:00:30] CHEGAD